# LLM-Informed Initialization for Bayesian Optimization

This notebook demonstrates how to use LLM-based initialization to get
**literature-informed starting points** before running Bayesian optimization.

Instead of starting with random experiments, we query an LLM with web search
capability to suggest initial conditions based on scientific literature and
best practices. This can significantly reduce the number of experiments needed
to find good conditions.

**Requirements:**
- `OPENAI_API_KEY` environment variable or `.env` file
- `python-dotenv` package (optional, for loading `.env`)

## Setup

Load the API key from environment and set up Folio.

In [ ]:
import os
import tempfile
import numpy as np
import matplotlib.pyplot as plt

# Load .env file if available
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Loaded .env file")
except ImportError:
    print("python-dotenv not installed, using existing environment variables")

# Check for API key
api_key = os.environ.get("OPENAI_API_KEY")
if api_key:
    print(f"API key found: {api_key[:8]}...{api_key[-4:]}")
else:
    raise ValueError("OPENAI_API_KEY not found in environment")

In [ ]:
from folio.api import Folio
from folio.core.config import TargetConfig
from folio.core.schema import InputSpec, OutputSpec
from folio.recommenders.initializer import OpenAIBackend

# Create temporary database
db_path = tempfile.mktemp(suffix=".db")
folio = Folio(db_path=db_path)
print(f"Database: {db_path}")

## Define a Realistic Optimization Problem

We'll optimize a **Suzuki coupling reaction** - a common palladium-catalyzed
cross-coupling reaction in organic synthesis.

Parameters:
- **temperature**: Reaction temperature (40-120°C)
- **catalyst_loading**: Pd catalyst mol% (0.5-5.0%)
- **solvent**: Choice of solvent (DMF, THF, dioxane, toluene)
- **base_equiv**: Equivalents of base (1.5-4.0)

Output:
- **yield**: Product yield (%)

For this demo, we'll simulate experiments with a synthetic function that has
its optimum around typical literature conditions.

In [ ]:
# Synthetic Suzuki coupling "experiment"
# Optimum around: 80°C, 2% Pd, dioxane, 2.5 equiv base

SOLVENT_EFFECTS = {
    "DMF": -5,
    "THF": -10,
    "dioxane": 0,  # Best
    "toluene": -15,
}

def suzuki_experiment(temperature: float, catalyst_loading: float, 
                      solvent: str, base_equiv: float) -> float:
    """Simulated Suzuki coupling yield.
    
    Optimal conditions: 80°C, 2% Pd, dioxane, 2.5 equiv base -> ~95% yield
    """
    # Temperature effect (bell curve around 80°C)
    temp_effect = -0.01 * (temperature - 80) ** 2
    
    # Catalyst loading (diminishing returns, optimum around 2%)
    cat_effect = -5 * (catalyst_loading - 2) ** 2
    
    # Solvent effect
    solvent_effect = SOLVENT_EFFECTS.get(solvent, -20)
    
    # Base equivalents (optimum around 2.5)
    base_effect = -3 * (base_equiv - 2.5) ** 2
    
    # Combine effects
    base_yield = 95
    noise = np.random.normal(0, 2)
    
    yield_value = base_yield + temp_effect + cat_effect + solvent_effect + base_effect + noise
    return max(0, min(100, yield_value))  # Clamp to [0, 100]

# Test it
print(f"Optimal conditions: {suzuki_experiment(80, 2.0, 'dioxane', 2.5):.1f}%")
print(f"Random conditions:  {suzuki_experiment(60, 4.0, 'THF', 3.5):.1f}%")

## Create the Project

In [ ]:
folio.create_project(
    name="suzuki_coupling",
    inputs=[
        InputSpec("temperature", "continuous", bounds=(40.0, 120.0), units="°C"),
        InputSpec("catalyst_loading", "continuous", bounds=(0.5, 5.0), units="mol%"),
        InputSpec("solvent", "categorical", levels=["DMF", "THF", "dioxane", "toluene"]),
        InputSpec("base_equiv", "continuous", bounds=(1.5, 4.0), units="equiv"),
    ],
    outputs=[OutputSpec("yield", units="%")],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)
print("Project 'suzuki_coupling' created!")

## LLM-Informed Initialization

Now we'll use the LLM initializer to get **literature-informed starting points**.

The LLM will:
1. Search the web for Suzuki coupling optimization papers
2. Suggest initial conditions based on literature best practices
3. Return validated suggestions that respect our parameter bounds

**Note:** This makes a real API call to OpenAI with web search. Cost is
typically ~$0.05-0.10 per call.

In [ ]:
# Create OpenAI backend (uses OPENAI_API_KEY from environment)
# Note: gpt-5.2 is the default, but gpt-4.1 works well and is cheaper
backend = OpenAIBackend(model="gpt-4.1")

# Get LLM suggestions with a descriptive problem statement
description = """
Optimizing a Suzuki-Miyaura cross-coupling reaction between an aryl halide
and a boronic acid. Looking for high yield conditions for a general substrate
scope. The reaction uses a palladium catalyst with an inorganic base.
"""

print("Querying LLM for literature-informed starting conditions...")
print("(This may take 10-30 seconds due to web search)\n")

try:
    llm_suggestions = folio.initialize_from_llm(
        project_name="suzuki_coupling",
        n=5,
        description=description,
        backend=backend,
        max_cost_per_call=0.20,
    )
    
    print(f"Received {len(llm_suggestions)} suggestions from LLM:\n")
    for i, suggestion in enumerate(llm_suggestions, 1):
        print(f"  {i}. T={suggestion['temperature']:.0f}°C, "
              f"Pd={suggestion['catalyst_loading']:.1f}%, "
              f"{suggestion['solvent']}, "
              f"base={suggestion['base_equiv']:.1f} equiv")
        
except Exception as e:
    print(f"LLM initialization failed: {e}")
    print("\nFalling back to random initialization...")
    llm_suggestions = None

## Run Initial Experiments

Execute the LLM-suggested experiments and record results.

In [ ]:
if llm_suggestions:
    print("Running LLM-suggested experiments:\n")
    
    for i, suggestion in enumerate(llm_suggestions, 1):
        # Run the experiment
        result = suzuki_experiment(
            temperature=suggestion["temperature"],
            catalyst_loading=suggestion["catalyst_loading"],
            solvent=suggestion["solvent"],
            base_equiv=suggestion["base_equiv"],
        )
        
        # Record observation
        folio.add_observation(
            project_name="suzuki_coupling",
            inputs=suggestion,
            outputs={"yield": result},
            tag="llm_init",
        )
        
        print(f"  Exp {i}: yield = {result:.1f}%")
    
    # Check initial performance
    obs = folio.get_observations("suzuki_coupling")
    yields = [o.outputs["yield"] for o in obs]
    print(f"\nBest from LLM initialization: {max(yields):.1f}%")
else:
    print("No LLM suggestions available, will start with random exploration")

## Continue with Bayesian Optimization

Now that we have LLM-informed starting points, continue optimization with BO.

In [ ]:
# Note: Current BO implementation handles continuous variables only.
# Categorical variables need to be handled separately (e.g., random sampling,
# or using the best solvent from LLM initialization).

import random

N_BO_ITERATIONS = 10
SOLVENTS = ["DMF", "THF", "dioxane", "toluene"]

# Find best solvent from LLM init to use as default
if llm_suggestions:
    best_llm_idx = max(range(len(llm_suggestions)), 
                       key=lambda i: folio.get_observations("suzuki_coupling")[i].outputs["yield"])
    best_solvent = llm_suggestions[best_llm_idx]["solvent"]
    print(f"Using best solvent from LLM init: {best_solvent}\n")
else:
    best_solvent = "dioxane"

print(f"Running {N_BO_ITERATIONS} BO iterations...\n")

for i in range(N_BO_ITERATIONS):
    # Get BO suggestion (continuous vars only)
    suggestion = folio.suggest("suzuki_coupling")[0]
    
    # Add categorical variable (use best from LLM init, or random)
    suggestion["solvent"] = best_solvent if random.random() > 0.3 else random.choice(SOLVENTS)
    
    # Run experiment
    result = suzuki_experiment(
        temperature=suggestion["temperature"],
        catalyst_loading=suggestion["catalyst_loading"],
        solvent=suggestion["solvent"],
        base_equiv=suggestion["base_equiv"],
    )
    
    # Record
    folio.add_observation(
        project_name="suzuki_coupling",
        inputs=suggestion,
        outputs={"yield": result},
        tag="bo",
    )
    
    print(f"  BO {i+1}: T={suggestion['temperature']:.0f}°C, "
          f"Pd={suggestion['catalyst_loading']:.1f}%, "
          f"{suggestion['solvent']}, "
          f"base={suggestion['base_equiv']:.1f} -> yield={result:.1f}%")

## Analyze Results

In [ ]:
# Get all observations
all_obs = folio.get_observations("suzuki_coupling")
llm_obs = [o for o in all_obs if o.tag == "llm_init"]
bo_obs = [o for o in all_obs if o.tag == "bo"]

all_yields = [o.outputs["yield"] for o in all_obs]
llm_yields = [o.outputs["yield"] for o in llm_obs]
bo_yields = [o.outputs["yield"] for o in bo_obs]

# Best result
best_idx = np.argmax(all_yields)
best_obs = all_obs[best_idx]

print("=" * 60)
print("OPTIMIZATION SUMMARY")
print("=" * 60)
print(f"\nTotal experiments: {len(all_obs)}")
print(f"  - LLM initialization: {len(llm_obs)}")
print(f"  - BO iterations: {len(bo_obs)}")
print(f"\nLLM initialization performance:")
print(f"  - Best: {max(llm_yields):.1f}%")
print(f"  - Mean: {np.mean(llm_yields):.1f}%")
print(f"\nFinal best result: {all_yields[best_idx]:.1f}%")
print(f"  Temperature: {best_obs.inputs['temperature']:.1f}°C")
print(f"  Catalyst: {best_obs.inputs['catalyst_loading']:.2f} mol%")
print(f"  Solvent: {best_obs.inputs['solvent']}")
print(f"  Base equiv: {best_obs.inputs['base_equiv']:.2f}")
print(f"\nTrue optimum: 95% at 80°C, 2% Pd, dioxane, 2.5 equiv")

In [ ]:
# Plot optimization progress
running_best = np.maximum.accumulate(all_yields)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Left: Optimization progress
n_llm = len(llm_yields) if llm_yields else 0
x = range(1, len(all_yields) + 1)

ax1.plot(x, running_best, "b-o", markersize=4, label="Best found")
ax1.axhline(95, color="r", linestyle="--", alpha=0.7, label="True optimum (95%)")
if n_llm > 0:
    ax1.axvline(n_llm + 0.5, color="gray", linestyle=":", alpha=0.7, label="LLM → BO transition")
    ax1.axvspan(0.5, n_llm + 0.5, alpha=0.1, color="green", label="LLM init")
ax1.set_xlabel("Experiment #")
ax1.set_ylabel("Yield (%)")
ax1.set_title("Optimization Progress")
ax1.legend(loc="lower right")
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 105)

# Right: Temperature vs Yield colored by iteration
temps = [o.inputs["temperature"] for o in all_obs]
colors = range(len(all_obs))
sc = ax2.scatter(temps, all_yields, c=colors, cmap="viridis", s=60)
ax2.axvline(80, color="r", linestyle="--", alpha=0.5, label="Optimal temp (80°C)")
ax2.set_xlabel("Temperature (°C)")
ax2.set_ylabel("Yield (%)")
ax2.set_title("Temperature vs Yield")
ax2.legend()
ax2.grid(True, alpha=0.3)
plt.colorbar(sc, ax=ax2, label="Experiment #")

plt.tight_layout()
plt.show()

## Comparison: LLM Init vs Random Init

To demonstrate the value of LLM-informed initialization, let's run a parallel
optimization starting from **random initial points** with the same total budget
(5 init + 10 BO = 15 experiments).

In [ ]:
# Create a separate project for random initialization comparison
folio.create_project(
    name="suzuki_random",
    inputs=[
        InputSpec("temperature", "continuous", bounds=(40.0, 120.0), units="°C"),
        InputSpec("catalyst_loading", "continuous", bounds=(0.5, 5.0), units="mol%"),
        InputSpec("solvent", "categorical", levels=["DMF", "THF", "dioxane", "toluene"]),
        InputSpec("base_equiv", "continuous", bounds=(1.5, 4.0), units="equiv"),
    ],
    outputs=[OutputSpec("yield", units="%")],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)

# Random initialization: 5 random experiments
print("Running RANDOM initialization (5 experiments):\n")
random_init_yields = []

for i in range(5):
    # Generate random inputs within bounds
    rand_inputs = {
        "temperature": np.random.uniform(40.0, 120.0),
        "catalyst_loading": np.random.uniform(0.5, 5.0),
        "solvent": random.choice(SOLVENTS),
        "base_equiv": np.random.uniform(1.5, 4.0),
    }
    
    result = suzuki_experiment(**rand_inputs)
    random_init_yields.append(result)
    
    folio.add_observation(
        project_name="suzuki_random",
        inputs=rand_inputs,
        outputs={"yield": result},
        tag="random_init",
    )
    
    print(f"  Exp {i+1}: T={rand_inputs['temperature']:.0f}°C, "
          f"Pd={rand_inputs['catalyst_loading']:.1f}%, "
          f"{rand_inputs['solvent']}, "
          f"base={rand_inputs['base_equiv']:.1f} -> yield={result:.1f}%")

print(f"\nBest from random init: {max(random_init_yields):.1f}%")
print(f"Mean from random init: {np.mean(random_init_yields):.1f}%")

In [ ]:
# Continue with BO for random-initialized project
print(f"Running {N_BO_ITERATIONS} BO iterations for random-init project...\n")

for i in range(N_BO_ITERATIONS):
    suggestion = folio.suggest("suzuki_random")[0]
    suggestion["solvent"] = random.choice(SOLVENTS)
    
    result = suzuki_experiment(**suggestion)
    
    folio.add_observation(
        project_name="suzuki_random",
        inputs=suggestion,
        outputs={"yield": result},
        tag="bo",
    )
    
    print(f"  BO {i+1}: T={suggestion['temperature']:.0f}°C, "
          f"Pd={suggestion['catalyst_loading']:.1f}%, "
          f"{suggestion['solvent']}, "
          f"base={suggestion['base_equiv']:.1f} -> yield={result:.1f}%")

In [ ]:
# Compare LLM init vs Random init
random_obs = folio.get_observations("suzuki_random")
random_yields = [o.outputs["yield"] for o in random_obs]
random_running_best = np.maximum.accumulate(random_yields)

# Get LLM results (already computed)
llm_running_best = np.maximum.accumulate(all_yields)

# Summary statistics
print("=" * 60)
print("COMPARISON: LLM Init vs Random Init")
print("=" * 60)
print(f"\n{'Metric':<30} {'LLM Init':>12} {'Random Init':>12}")
print("-" * 54)
print(f"{'Best after 5 init experiments:':<30} {max(llm_yields):>11.1f}% {max(random_init_yields):>11.1f}%")
print(f"{'Mean of init experiments:':<30} {np.mean(llm_yields):>11.1f}% {np.mean(random_init_yields):>11.1f}%")
print(f"{'Final best (15 experiments):':<30} {max(all_yields):>11.1f}% {max(random_yields):>11.1f}%")
print(f"{'Experiments to reach 85%:':<30}", end="")

# Find first experiment to reach 85%
llm_first_85 = next((i+1 for i, y in enumerate(llm_running_best) if y >= 85), ">15")
rand_first_85 = next((i+1 for i, y in enumerate(random_running_best) if y >= 85), ">15")
print(f" {llm_first_85:>11} {rand_first_85:>11}")

In [ ]:
# Plot comparison
fig, ax = plt.subplots(figsize=(10, 5))

x = range(1, 16)
ax.plot(x, llm_running_best, "g-o", markersize=6, linewidth=2, label="LLM Init + BO")
ax.plot(x, random_running_best, "b-s", markersize=6, linewidth=2, label="Random Init + BO")
ax.axhline(95, color="r", linestyle="--", alpha=0.7, label="True optimum (95%)")
ax.axvline(5.5, color="gray", linestyle=":", alpha=0.5, label="Init → BO transition")

# Shade init regions
ax.axvspan(0.5, 5.5, alpha=0.1, color="green")
ax.text(3, 50, "Init\nPhase", ha="center", fontsize=10, alpha=0.7)
ax.text(10, 50, "BO Phase", ha="center", fontsize=10, alpha=0.7)

ax.set_xlabel("Experiment #", fontsize=12)
ax.set_ylabel("Best Yield Found (%)", fontsize=12)
ax.set_title("LLM-Informed vs Random Initialization", fontsize=14)
ax.legend(loc="lower right")
ax.grid(True, alpha=0.3)
ax.set_xlim(0.5, 15.5)
ax.set_ylim(0, 105)

plt.tight_layout()
plt.show()

# Cleanup random project
folio.delete_project("suzuki_random")

## Key Takeaways

1. **LLM-informed initialization** provides reasonable starting points based on
   literature knowledge, often better than random exploration.

2. **Web search capability** allows the LLM to find relevant papers and extract
   typical reaction conditions for your specific chemistry.

3. **Seamless integration** - LLM suggestions are validated against your project
   schema (bounds checking, categorical levels) before use.

4. **Cost control** - The `max_cost_per_call` parameter prevents unexpected
   API charges. Typical calls cost $0.05-0.10.

5. **Graceful fallback** - If LLM initialization fails, you can fall back to
   random initialization or manually specify starting conditions.

6. **Hybrid approach** - Combine LLM domain knowledge with BO's data-driven
   optimization for best results.

7. **Categorical variables** - Current BO implementation handles continuous
   variables only. Categorical variables can be handled by using the best
   value from LLM initialization or random sampling. KABO embeddings (planned
   for v1.1) will enable proper categorical optimization.

In [ ]:
# Cleanup
folio.delete_project("suzuki_coupling")
print("Demo complete!")

In [ ]:
import os
import tempfile
import numpy as np
import matplotlib.pyplot as plt

# Load .env file if available
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Loaded .env file")
except ImportError:
    print("python-dotenv not installed, using existing environment variables")

# Check for API key
api_key = os.environ.get("OPENAI_API_KEY")
if api_key:
    print(f"API key found: {api_key[:8]}...{api_key[-4:]}")
else:
    raise ValueError("OPENAI_API_KEY not found in environment")

In [ ]:
from folio.api import Folio
from folio.core.config import TargetConfig
from folio.core.schema import InputSpec, OutputSpec
from folio.recommenders.initializer import OpenAIBackend

# Create temporary database
db_path = tempfile.mktemp(suffix=".db")
folio = Folio(db_path=db_path)
print(f"Database: {db_path}")

In [ ]:
# Synthetic Suzuki coupling "experiment"
# Optimum around: 80°C, 2% Pd, dioxane, 2.5 equiv base

SOLVENT_EFFECTS = {
    "DMF": -5,
    "THF": -10,
    "dioxane": 0,  # Best
    "toluene": -15,
}

def suzuki_experiment(temperature: float, catalyst_loading: float, 
                      solvent: str, base_equiv: float) -> float:
    """Simulated Suzuki coupling yield.
    
    Optimal conditions: 80°C, 2% Pd, dioxane, 2.5 equiv base -> ~95% yield
    """
    # Temperature effect (bell curve around 80°C)
    temp_effect = -0.01 * (temperature - 80) ** 2
    
    # Catalyst loading (diminishing returns, optimum around 2%)
    cat_effect = -5 * (catalyst_loading - 2) ** 2
    
    # Solvent effect
    solvent_effect = SOLVENT_EFFECTS.get(solvent, -20)
    
    # Base equivalents (optimum around 2.5)
    base_effect = -3 * (base_equiv - 2.5) ** 2
    
    # Combine effects
    base_yield = 95
    noise = np.random.normal(0, 2)
    
    yield_value = base_yield + temp_effect + cat_effect + solvent_effect + base_effect + noise
    return max(0, min(100, yield_value))  # Clamp to [0, 100]

# Test it
print(f"Optimal conditions: {suzuki_experiment(80, 2.0, 'dioxane', 2.5):.1f}%")
print(f"Random conditions:  {suzuki_experiment(60, 4.0, 'THF', 3.5):.1f}%")

In [ ]:
folio.create_project(
    name="suzuki_coupling",
    inputs=[
        InputSpec("temperature", "continuous", bounds=(40.0, 120.0), units="°C"),
        InputSpec("catalyst_loading", "continuous", bounds=(0.5, 5.0), units="mol%"),
        InputSpec("solvent", "categorical", levels=["DMF", "THF", "dioxane", "toluene"]),
        InputSpec("base_equiv", "continuous", bounds=(1.5, 4.0), units="equiv"),
    ],
    outputs=[OutputSpec("yield", units="%")],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)
print("Project 'suzuki_coupling' created!")

In [ ]:
# Create OpenAI backend (uses OPENAI_API_KEY from environment)
# Note: gpt-5.2 is the default, but gpt-4.1 works well and is cheaper
backend = OpenAIBackend(model="gpt-4.1")

# Get LLM suggestions with a descriptive problem statement
description = """
Optimizing a Suzuki-Miyaura cross-coupling reaction between an aryl halide
and a boronic acid. Looking for high yield conditions for a general substrate
scope. The reaction uses a palladium catalyst with an inorganic base.
"""

print("Querying LLM for literature-informed starting conditions...")
print("(This may take 10-30 seconds due to web search)\n")

try:
    llm_suggestions = folio.initialize_from_llm(
        project_name="suzuki_coupling",
        n=5,
        description=description,
        backend=backend,
        max_cost_per_call=0.20,
    )
    
    print(f"Received {len(llm_suggestions)} suggestions from LLM:\n")
    for i, suggestion in enumerate(llm_suggestions, 1):
        print(f"  {i}. T={suggestion['temperature']:.0f}°C, "
              f"Pd={suggestion['catalyst_loading']:.1f}%, "
              f"{suggestion['solvent']}, "
              f"base={suggestion['base_equiv']:.1f} equiv")
        
except Exception as e:
    print(f"LLM initialization failed: {e}")
    print("\nFalling back to random initialization...")
    llm_suggestions = None

In [ ]:
if llm_suggestions:
    print("Running LLM-suggested experiments:\n")
    
    for i, suggestion in enumerate(llm_suggestions, 1):
        # Run the experiment
        result = suzuki_experiment(
            temperature=suggestion["temperature"],
            catalyst_loading=suggestion["catalyst_loading"],
            solvent=suggestion["solvent"],
            base_equiv=suggestion["base_equiv"],
        )
        
        # Record observation
        folio.add_observation(
            project_name="suzuki_coupling",
            inputs=suggestion,
            outputs={"yield": result},
            tag="llm_init",
        )
        
        print(f"  Exp {i}: yield = {result:.1f}%")
    
    # Check initial performance
    obs = folio.get_observations("suzuki_coupling")
    yields = [o.outputs["yield"] for o in obs]
    print(f"\nBest from LLM initialization: {max(yields):.1f}%")
else:
    print("No LLM suggestions available, will start with random exploration")

In [ ]:
N_BO_ITERATIONS = 10

print(f"Running {N_BO_ITERATIONS} BO iterations...\n")

for i in range(N_BO_ITERATIONS):
    # Get BO suggestion
    suggestion = folio.suggest("suzuki_coupling")[0]
    
    # Run experiment
    result = suzuki_experiment(
        temperature=suggestion["temperature"],
        catalyst_loading=suggestion["catalyst_loading"],
        solvent=suggestion["solvent"],
        base_equiv=suggestion["base_equiv"],
    )
    
    # Record
    folio.add_observation(
        project_name="suzuki_coupling",
        inputs=suggestion,
        outputs={"yield": result},
        tag="bo",
    )
    
    print(f"  BO {i+1}: T={suggestion['temperature']:.0f}°C, "
          f"Pd={suggestion['catalyst_loading']:.1f}%, "
          f"{suggestion['solvent']}, "
          f"base={suggestion['base_equiv']:.1f} -> yield={result:.1f}%")

In [ ]:
# Debug: check what suggestion looks like
suggestion = folio.suggest("suzuki_coupling")[0]
print("Suggestion:", suggestion)
print("Keys:", list(suggestion.keys()))

In [ ]:
# Workaround: BO doesn't return categorical variables, so we handle it manually
import random

N_BO_ITERATIONS = 10
SOLVENTS = ["DMF", "THF", "dioxane", "toluene"]

print(f"Running {N_BO_ITERATIONS} BO iterations...")
print("(Note: categorical 'solvent' is sampled randomly since BO handles continuous vars only)\n")

for i in range(N_BO_ITERATIONS):
    # Get BO suggestion (continuous vars only)
    suggestion = folio.suggest("suzuki_coupling")[0]
    
    # Add categorical variable (random for now, or could use best from LLM init)
    suggestion["solvent"] = random.choice(SOLVENTS)
    
    # Run experiment
    result = suzuki_experiment(
        temperature=suggestion["temperature"],
        catalyst_loading=suggestion["catalyst_loading"],
        solvent=suggestion["solvent"],
        base_equiv=suggestion["base_equiv"],
    )
    
    # Record
    folio.add_observation(
        project_name="suzuki_coupling",
        inputs=suggestion,
        outputs={"yield": result},
        tag="bo",
    )
    
    print(f"  BO {i+1}: T={suggestion['temperature']:.0f}°C, "
          f"Pd={suggestion['catalyst_loading']:.1f}%, "
          f"{suggestion['solvent']}, "
          f"base={suggestion['base_equiv']:.1f} -> yield={result:.1f}%")

In [ ]:
# Get all observations
all_obs = folio.get_observations("suzuki_coupling")
llm_obs = [o for o in all_obs if o.tag == "llm_init"]
bo_obs = [o for o in all_obs if o.tag == "bo"]

all_yields = [o.outputs["yield"] for o in all_obs]
llm_yields = [o.outputs["yield"] for o in llm_obs]
bo_yields = [o.outputs["yield"] for o in bo_obs]

# Best result
best_idx = np.argmax(all_yields)
best_obs = all_obs[best_idx]

print("=" * 60)
print("OPTIMIZATION SUMMARY")
print("=" * 60)
print(f"\nTotal experiments: {len(all_obs)}")
print(f"  - LLM initialization: {len(llm_obs)}")
print(f"  - BO iterations: {len(bo_obs)}")
print(f"\nLLM initialization performance:")
print(f"  - Best: {max(llm_yields):.1f}%")
print(f"  - Mean: {np.mean(llm_yields):.1f}%")
print(f"\nFinal best result: {all_yields[best_idx]:.1f}%")
print(f"  Temperature: {best_obs.inputs['temperature']:.1f}°C")
print(f"  Catalyst: {best_obs.inputs['catalyst_loading']:.2f} mol%")
print(f"  Solvent: {best_obs.inputs['solvent']}")
print(f"  Base equiv: {best_obs.inputs['base_equiv']:.2f}")
print(f"\nTrue optimum: 95% at 80°C, 2% Pd, dioxane, 2.5 equiv")

In [ ]:
# Plot optimization progress
running_best = np.maximum.accumulate(all_yields)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Left: Optimization progress
n_llm = len(llm_yields) if llm_yields else 0
x = range(1, len(all_yields) + 1)

ax1.plot(x, running_best, "b-o", markersize=4, label="Best found")
ax1.axhline(95, color="r", linestyle="--", alpha=0.7, label="True optimum (95%)")
if n_llm > 0:
    ax1.axvline(n_llm + 0.5, color="gray", linestyle=":", alpha=0.7, label="LLM → BO transition")
    ax1.axvspan(0.5, n_llm + 0.5, alpha=0.1, color="green", label="LLM init")
ax1.set_xlabel("Experiment #")
ax1.set_ylabel("Yield (%)")
ax1.set_title("Optimization Progress")
ax1.legend(loc="lower right")
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 105)

# Right: Temperature vs Yield colored by iteration
temps = [o.inputs["temperature"] for o in all_obs]
colors = range(len(all_obs))
sc = ax2.scatter(temps, all_yields, c=colors, cmap="viridis", s=60)
ax2.axvline(80, color="r", linestyle="--", alpha=0.5, label="Optimal temp (80°C)")
ax2.set_xlabel("Temperature (°C)")
ax2.set_ylabel("Yield (%)")
ax2.set_title("Temperature vs Yield")
ax2.legend()
ax2.grid(True, alpha=0.3)
plt.colorbar(sc, ax=ax2, label="Experiment #")

plt.tight_layout()
plt.show()

In [ ]:
# Cleanup
folio.delete_project("suzuki_coupling")
print("Demo complete!")

In [ ]:
# Re-setup for comparison (since we cleaned up earlier)
import os
import tempfile
import numpy as np
import random
import matplotlib.pyplot as plt

from dotenv import load_dotenv
load_dotenv()

from folio.api import Folio
from folio.core.config import TargetConfig
from folio.core.schema import InputSpec, OutputSpec
from folio.recommenders.initializer import OpenAIBackend

db_path = tempfile.mktemp(suffix=".db")
folio = Folio(db_path=db_path)

SOLVENT_EFFECTS = {"DMF": -5, "THF": -10, "dioxane": 0, "toluene": -15}
SOLVENTS = ["DMF", "THF", "dioxane", "toluene"]

def suzuki_experiment(temperature, catalyst_loading, solvent, base_equiv):
    temp_effect = -0.01 * (temperature - 80) ** 2
    cat_effect = -5 * (catalyst_loading - 2) ** 2
    solvent_effect = SOLVENT_EFFECTS.get(solvent, -20)
    base_effect = -3 * (base_equiv - 2.5) ** 2
    noise = np.random.normal(0, 2)
    yield_value = 95 + temp_effect + cat_effect + solvent_effect + base_effect + noise
    return max(0, min(100, yield_value))

# Create both projects
for name in ["suzuki_llm", "suzuki_random"]:
    folio.create_project(
        name=name,
        inputs=[
            InputSpec("temperature", "continuous", bounds=(40.0, 120.0), units="°C"),
            InputSpec("catalyst_loading", "continuous", bounds=(0.5, 5.0), units="mol%"),
            InputSpec("solvent", "categorical", levels=SOLVENTS),
            InputSpec("base_equiv", "continuous", bounds=(1.5, 4.0), units="equiv"),
        ],
        outputs=[OutputSpec("yield", units="%")],
        target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
    )

print("Projects created. Ready for comparison.")

In [ ]:
# LLM Initialization
backend = OpenAIBackend(model="gpt-4.1")
description = "Optimizing a Suzuki-Miyaura cross-coupling reaction with Pd catalyst."

print("=" * 60)
print("LLM INITIALIZATION")
print("=" * 60)
print("\nQuerying LLM for literature-informed conditions...")

llm_suggestions = folio.initialize_from_llm(
    project_name="suzuki_llm",
    n=5,
    description=description,
    backend=backend,
    max_cost_per_call=0.20,
)

print(f"\nReceived {len(llm_suggestions)} LLM suggestions:")
llm_yields = []
for i, s in enumerate(llm_suggestions, 1):
    result = suzuki_experiment(**s)
    llm_yields.append(result)
    folio.add_observation("suzuki_llm", inputs=s, outputs={"yield": result}, tag="init")
    print(f"  {i}. T={s['temperature']:.0f}°C, Pd={s['catalyst_loading']:.1f}%, "
          f"{s['solvent']}, base={s['base_equiv']:.1f} -> {result:.1f}%")

print(f"\nLLM Init: Best={max(llm_yields):.1f}%, Mean={np.mean(llm_yields):.1f}%")

In [ ]:
# Random Initialization
print("\n" + "=" * 60)
print("RANDOM INITIALIZATION")
print("=" * 60)

random_yields = []
for i in range(5):
    rand_inputs = {
        "temperature": np.random.uniform(40.0, 120.0),
        "catalyst_loading": np.random.uniform(0.5, 5.0),
        "solvent": random.choice(SOLVENTS),
        "base_equiv": np.random.uniform(1.5, 4.0),
    }
    result = suzuki_experiment(**rand_inputs)
    random_yields.append(result)
    folio.add_observation("suzuki_random", inputs=rand_inputs, outputs={"yield": result}, tag="init")
    print(f"  {i+1}. T={rand_inputs['temperature']:.0f}°C, Pd={rand_inputs['catalyst_loading']:.1f}%, "
          f"{rand_inputs['solvent']}, base={rand_inputs['base_equiv']:.1f} -> {result:.1f}%")

print(f"\nRandom Init: Best={max(random_yields):.1f}%, Mean={np.mean(random_yields):.1f}%")

In [ ]:
# Run BO for both projects
N_BO = 10

print("\n" + "=" * 60)
print("BAYESIAN OPTIMIZATION (10 iterations each)")
print("=" * 60)

for project_name, label in [("suzuki_llm", "LLM+BO"), ("suzuki_random", "Rand+BO")]:
    print(f"\n{label}:")
    for i in range(N_BO):
        suggestion = folio.suggest(project_name)[0]
        suggestion["solvent"] = random.choice(SOLVENTS)
        result = suzuki_experiment(**suggestion)
        folio.add_observation(project_name, inputs=suggestion, outputs={"yield": result}, tag="bo")
        print(f"  {i+1}. T={suggestion['temperature']:.0f}°C -> {result:.1f}%")

In [ ]:
# Comparison summary
llm_obs = folio.get_observations("suzuki_llm")
rand_obs = folio.get_observations("suzuki_random")

llm_all_yields = [o.outputs["yield"] for o in llm_obs]
rand_all_yields = [o.outputs["yield"] for o in rand_obs]

llm_running_best = np.maximum.accumulate(llm_all_yields)
rand_running_best = np.maximum.accumulate(rand_all_yields)

llm_init_yields = [o.outputs["yield"] for o in llm_obs if o.tag == "init"]
rand_init_yields = [o.outputs["yield"] for o in rand_obs if o.tag == "init"]

print("\n" + "=" * 60)
print("COMPARISON: LLM Init vs Random Init")
print("=" * 60)
print(f"\n{'Metric':<35} {'LLM Init':>10} {'Random':>10}")
print("-" * 57)
print(f"{'Best after 5 init experiments:':<35} {max(llm_init_yields):>9.1f}% {max(rand_init_yields):>9.1f}%")
print(f"{'Mean of init experiments:':<35} {np.mean(llm_init_yields):>9.1f}% {np.mean(rand_init_yields):>9.1f}%")
print(f"{'Final best (15 experiments):':<35} {max(llm_all_yields):>9.1f}% {max(rand_all_yields):>9.1f}%")

# Find first experiment to reach thresholds
for threshold in [80, 85]:
    llm_first = next((i+1 for i, y in enumerate(llm_running_best) if y >= threshold), ">15")
    rand_first = next((i+1 for i, y in enumerate(rand_running_best) if y >= threshold), ">15")
    print(f"{'Experiments to reach ' + str(threshold) + '%:':<35} {str(llm_first):>10} {str(rand_first):>10}")

In [ ]:
# Quick comparison summary
print("=" * 60)
print("COMPARISON: LLM Init vs Random Init")
print("=" * 60)
print(f"\n{'Metric':<35} {'LLM Init':>10} {'Random':>10}")
print("-" * 57)
print(f"{'Best after 5 init experiments:':<35} {max(llm_yields):>9.1f}% {max(random_yields):>9.1f}%")
print(f"{'Mean of init experiments:':<35} {np.mean(llm_yields):>9.1f}% {np.mean(random_yields):>9.1f}%")

# Get final results
llm_obs = folio.get_observations("suzuki_llm")
rand_obs = folio.get_observations("suzuki_random")
llm_all = [o.outputs["yield"] for o in llm_obs]
rand_all = [o.outputs["yield"] for o in rand_obs]

print(f"{'Final best (15 experiments):':<35} {max(llm_all):>9.1f}% {max(rand_all):>9.1f}%")
print(f"\nAdvantage of LLM init: {max(llm_yields) - max(random_yields):.1f}% better start")

In [ ]:
# Comparison plot
llm_running = np.maximum.accumulate(llm_all)
rand_running = np.maximum.accumulate(rand_all)

fig, ax = plt.subplots(figsize=(10, 5))

x = range(1, 16)
ax.plot(x, llm_running, "g-o", markersize=6, linewidth=2, label="LLM Init + BO")
ax.plot(x, rand_running, "b-s", markersize=6, linewidth=2, label="Random Init + BO")
ax.axhline(95, color="r", linestyle="--", alpha=0.7, label="True optimum (95%)")
ax.axvline(5.5, color="gray", linestyle=":", alpha=0.5)

ax.axvspan(0.5, 5.5, alpha=0.1, color="green")
ax.text(3, 45, "Init\nPhase", ha="center", fontsize=10, alpha=0.7)
ax.text(10.5, 45, "BO Phase", ha="center", fontsize=10, alpha=0.7)

ax.set_xlabel("Experiment #", fontsize=12)
ax.set_ylabel("Best Yield Found (%)", fontsize=12)
ax.set_title("LLM-Informed vs Random Initialization", fontsize=14)
ax.legend(loc="lower right")
ax.grid(True, alpha=0.3)
ax.set_xlim(0.5, 15.5)
ax.set_ylim(40, 100)

plt.tight_layout()
plt.show()

# Cleanup
folio.delete_project("suzuki_llm")
folio.delete_project("suzuki_random")
print("\nComparison complete!")